##### Тестирование всех фичей с рассчитанными фичами в витрине ЦАЛ 

Перед этим необходимо подготовить тестовые данные согласно prepare_test_data.ipynb 
и сохранить их в папку data/interm/test 

In [14]:
pre = '../../' # путь до главной директории проекта
%run ../../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)
%run ../init.ipynb #подгрузить нужные библиотеки

In [15]:
# сюда таблицы служебные будем кидать, для отчетов
name = pre + "data/interm/auxilary/test_features.xlsx"
writer = ExcelWriter(name)

In [16]:
name = pre + "data/interm/test/target_features.pkl"
target_features = pd.read_pickle(name, compression={'method':'zip'})

print(len(target_features))
# target_features.head(1)

87180


In [17]:
name = pre + "data/external/model_tsal_for_test.pkl"
model_tsal_prod = pd.read_pickle(name, compression={'method':'zip'})

print(len(model_tsal_prod))
# model_tsal_prod.head(1)

156409


In [18]:
print(model_tsal_prod['DAILY_SNAPSHOT_DT'].min())
print(model_tsal_prod['DAILY_SNAPSHOT_DT'].max())

2023-07-04 00:00:00
2023-07-05 00:00:00


In [19]:
# берем последний срез
model_tsal_prod = model_tsal_prod[model_tsal_prod['DAILY_SNAPSHOT_DT']==model_tsal_prod['DAILY_SNAPSHOT_DT'].max()]

num, duplicates = identify_duplicates(model_tsal_prod, 'NEW_INN')
Check_dupl(num, duplicates)

In [20]:
kwargs = dict( sourcesName = ['tsal', 'target'], 
               suffix = '_target',
               left_on = 'NEW_INN', right_on = 'ИНН контрагента', 
               notnull_col = ['DAILY_SNAPSHOT_DT', 'PL'],
               base_col = ['NEW_INN', 'Номер договора (кратко)']
             )

comp =  ComparisonDataFrames(model_tsal_prod, target_features, **kwargs)   

In [21]:
missed = comp.missedRows(sourcesName='tsal')

отсутствуют в tsal 19629 строк


отсутствие в ЦАЛ связано с тем, что были исключены крупные клиенты

In [22]:
df = comp.presentRows()
# df.head(1)

одновременно присутствуют в tsal и target 67551 строк


In [23]:
df = comp.check_fields('CONTRACT_CNT', 'contract_cnt')
# df.head(1)

значения не совпадают у 12 строк из 67551


In [27]:
add_condition = (comp.inner_merge['DSNC_OVD_PAY_DAYS'].notnull())

df = comp.check_fields('DSNC_OVD_PAY_DAYS', 'dsnc_ovd_pay_days', add_condition = add_condition)
# df.head()


значения не совпадают у 31 строк из 67551
с учетом ограничения проверялось 48353 строк


In [28]:
add_condition = (comp.inner_merge['MAX_LEAS_OVD_PAY_DT'].notnull())

df = comp.check_fields('MAX_LEAS_OVD_PAY_DT', 'max_leas_ovd_pay_dt', add_condition = add_condition)
# df.head()


значения не совпадают у 31 строк из 67551
с учетом ограничения проверялось 48353 строк
